### Working With Databases ###


Lets create a database using sqlite3.
```
$ sqlite3 a.db
SQLite version 3.8.10.2 2015-05-20 18:17:19
Enter ".help" for usage hints.
sqlite> create table person (name varchar(100), email varchar(100));
sqlite>
sqlite> select * from person;
sqlite> insert into person (name, email) values ('alice', 'alice@example.com');
sqlite> select * from person;
alice|alice@example.com
```


In [3]:
import sqlite3

In [5]:
conn = sqlite3.connect("data.db")
cur = conn.cursor()
result = cur.execute("select * from person")
result.fetchall()

[('alice', 'alice@wonder.land')]

In [7]:
# WARNING: not a good implementation
def find_person(email):
    q = "select * from person where email='{}'".format(email)
    print(q)
    cur = conn.cursor()
    result = cur.execute(q)
    return result.fetchone()

find_person("alice@wonder.land")

select * from person where email='alice@wonder.land'


('alice', 'alice@wonder.land')

In [11]:
def find_person(email):
    query = "select * from person where email=?"
    cur = conn.cursor()t
    result = cur.execute(query, (email,))
    return result.fetchone()
    

In [12]:
find_person("alice@wonder.land")

('alice', 'alice@wonder.land')

In [13]:
def query(conn, querystring, params=()):
    cur = conn.cursor()
    result = cur.execute(querystring, params)
    return result.fetchall()

In [14]:
query(conn, "select * from person where name=?", ("alice",))

[('alice', 'alice@wonder.land')]

In [15]:
query(conn, "select * from person")

[('alice', 'alice@wonder.land')]

In [16]:
conn.close()

In [17]:
conn = sqlite3.connect("data.db")

In [4]:
cur = conn.cursor()
persons = [("dilbert", "dilbert@dilbert.com"),
          ("calvin", "calvin@calvinhobes.com"),
          ("jerry", "jerry@disney.com")]
cur.executemany("insert into person values(?,?)", persons)

In [5]:
conn.commit() # this will save the changes to db file
conn.close()

In [4]:
conn = sqlite3.connect("data.db")
cur = conn.cursor()
for name, email in cur.execute("select * from person ORDER by name"):
    print(name, email)

alice alice@wonder.land
calvin calvin@calvinhobes.com
dilbert dilbert@dilbert.com
jerry jerry@disney.com


### sqlalchemy ###

In [1]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, create_engine

metadata = MetaData()

users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('email', String))

posts = Table('posts', metadata,
    Column('id', Integer, primary_key=True),
    Column('author_id', Integer, ForeignKey('users.id')),
    Column('title', String),
    Column('body', String))

In [2]:
engine = create_engine('sqlite:///test.db', echo=True)
metadata.create_all(engine)

2017-11-14 21:22:23,502 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-11-14 21:22:23,504 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:22:23,511 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-11-14 21:22:23,516 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:22:23,523 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-11-14 21:22:23,527 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:22:23,535 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("posts")
2017-11-14 21:22:23,538 INFO sqlalchemy.engine.base.Engine ()


In [3]:
q = users.insert().values(name="Alice",email="alice@wonder.land")
# connect the engine to the metadata
metadata.bind = engine
q.execute()

2017-11-14 21:22:29,285 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, email) VALUES (?, ?)
2017-11-14 21:22:29,291 INFO sqlalchemy.engine.base.Engine ('Alice', 'alice@wonder.land')
2017-11-14 21:22:29,298 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
result = users.select().execute()

2017-11-14 21:23:52,777 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.email 
FROM users
2017-11-14 21:23:52,784 INFO sqlalchemy.engine.base.Engine ()


In [6]:
result.fetchall()

[(1, 'Alice', 'alice@wonder.land'), (2, 'Alice', 'alice@wonder.land')]

In [8]:
select = users.select()

In [10]:
q = select.where(users.c.email=="alice@wonder.land")

In [11]:
print(q)

SELECT users.id, users.name, users.email 
FROM users 
WHERE users.email = ?


In [13]:
print(select)

SELECT users.id, users.name, users.email 
FROM users


In [19]:
select.where(users.columns.name=="Alice").execute().fetchall()

2017-11-14 21:32:22,629 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.name = ?
2017-11-14 21:32:22,635 INFO sqlalchemy.engine.base.Engine ('Alice',)


[(1, 'Alice', 'alice@wonder.land'), (2, 'Alice', 'alice@wonder.land')]

** Q: How to work with existing db? **

In [23]:
engine = create_engine("sqlite:///test.db",echo=True)
metadata = MetaData(engine)

In [24]:
metadata.tables

immutabledict({})

In [25]:
metadata.reflect() # loads all the existing tables

2017-11-14 21:46:36,157 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-11-14 21:46:36,160 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:46:36,163 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-11-14 21:46:36,165 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:46:36,167 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2017-11-14 21:46:36,169 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:46:36,172 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("posts")
2017-11-14 21:46:36,173 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:46:36,175 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'posts' AND type = 'table'
2017-11-14 21:46:36,176 INFO sqlalchemy.engine.base.Engine ()
2017-11-14 21:46:36,178 INFO sqlalchemy.engine.base.

In [26]:
metadata.tables

immutabledict({'posts': Table('posts', MetaData(bind=Engine(sqlite:///test.db)), Column('id', INTEGER(), table=<posts>, primary_key=True, nullable=False), Column('author_id', INTEGER(), ForeignKey('users.id'), table=<posts>), Column('title', VARCHAR(), table=<posts>), Column('body', VARCHAR(), table=<posts>), schema=None), 'users': Table('users', MetaData(bind=Engine(sqlite:///test.db)), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<users>), Column('email', VARCHAR(), table=<users>), schema=None)})

### Objects-Relational Mapping (ORM) ###

In [31]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
engine = create_engine('sqlite:///test.db')
Base = declarative_base()

# connect the engine so that we don't have to worry about it later
Base.metadata.bind = engine 

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    
    def __repr__(self):
        return "User({0},{1})".format(self.name, self.email)

class Post(Base):
    __tablename__ = "posts"
    id = Column(Integer, primary_key=True)
    author_id = Column(Integer, ForeignKey("users.id"))
    title = Column(String)
    body = Column(String)
    
    def __repr__(self):
        return "Post({author},{title})".format(author=self.author_id, title=self.title)
    

from sqlalchemy.orm import sessionmaker
DBSession = sessionmaker(bind=engine)
session = DBSession()

new_user = User(name="anand",email="anand@anandology.com")
session.add(new_user)
session.commit()

In [32]:
q = session.query(User)
print(q.all())

[User(Alice,alice@wonder.land), User(Alice,alice@wonder.land), User(anand,anand@anandology.com), User(anand,anand@anandology.com)]


In [34]:
q.filter(User.name=="anand").all()

[User(anand,anand@anandology.com), User(anand,anand@anandology.com)]

#### Applying joins ####

In [35]:
q = session.query(User, Post)

In [36]:
print(q)

SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email, posts.id AS posts_id, posts.author_id AS posts_author_id, posts.title AS posts_title, posts.body AS posts_body 
FROM users, posts
